### IMPORTANT: Replace ```?``` in the following code with your uniqname.

In [8]:
MY_UNIQNAME = '?'

## Before we start...
## <font color="magenta">Q0: (2 points) Please let us know what you found confusing in the last class. </font>
We'll try to take time in the next class to review these concepts next class.

Insert your answer here.

# Loading and manipulating data in pandas

## Learning Objectives
* load CSV files
* load JSON files
* use pd.read_html to extract tables from web pages
* handle missing data (dropna and fillna)
* use vectorized string functions
* use Pandas' apply function to run a function on each row of a dataframe
* view and set the indexes of a dataframe, including hierarchical indexes
* use loc to explore a dataframe with hierarchical indexes
* use stack and unstack to reshape dataframes
* concatenate two DataFrames by columns
* use Pandas' merge function to join dataframes in a SQL-like way
* use the .describe() function
* understand .groupby()
* know how to use pivot and pivot_table

In [9]:
import pandas as pd

Recall the ```pd.read_csv``` function that we used to load data sets in previous classes:

In [10]:
menu = pd.read_csv('data/titles.csv') 

That works great for well-formatted CSV files, but what happens when you get something that looks like the ```data/avocado_eu.csv``` file.

If you're using JupyterLab, go ahead and browse that in JupyterLab's CSV browser.  You'll notice a new drop-down menu labelled "Delimiter".  Go ahead and change that to ```;```.

Referring back to your readings and the [read_csv documentation online](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html), complete the following exercise

Read the data/avocado_eu.csv file into a pandas DataFrame and show the first 5 rows.


In [11]:
avocado = pd.read_csv('data/avocado_eu.csv', sep=';', decimal=',')
avocado.head(5)

,Unnamed: 0,Date,AveragePrice,Total Volume,4046,4225,4770,Total Bags,Small Bags,Large Bags,XLarge Bags,type,year,region
0,0,2015-12-27,1.33,64236.62,1036.74,54454.85,48.16,8696.87,8603.62,93.25,0.0,conventional,2015,Albany
1,1,2015-12-20,1.35,54876.98,674.28,44638.81,58.33,9505.56,9408.07,97.49,0.0,conventional,2015,Albany
2,2,2015-12-13,0.93,118220.22,794.70,109149.67,130.50,8145.35,8042.21,103.14,0.0,conventional,2015,Albany
3,3,2015-12-06,1.08,78992.15,1132.00,71976.41,72.58,5811.16,5677.40,133.76,0.0,conventional,2015,Albany
4,4,2015-11-29,1.28,51039.60,941.48,43838.39,75.78,6183.95,5986.26,197.69,0.0,conventional,2015,Albany


You'll notice that, unless you did something special in the previous read_csv invocation, the decimal points don't look quite right.  Go ahead and find the right option to convert commas to periods when loading a CSV file.

## <font color="magenta">Q1 (2 points): 
Read the data/avocado_eu.csv file using the correct delimiter and decimal character into a dataframe and show the first 5 rows: </font>

In [12]:
#Insert your code here

## Counting the number of values (review)

Sometimes, you'll want to count the number of times values occur.  For example, we might want to know the number of times each 'type'
is reported in our avocado data.  Use the ```value_counts()``` function on a Series to do so:

In [13]:
avocado['type'].value_counts()

conventional    9126
organic         9123
Name: type, dtype: int64

## Loading JSON data

In addition to CSV files, JSON (JavaScript Object Notation) files or data is commonly used.  

In [14]:
nfl_football_players = pd.read_json('data/nfl_football_profiles.json')

In [15]:
nfl_football_players.head()

,player_id,name,position,height,weight,current_team,birth_date,birth_place,death_date,college,high_school,draft_team,draft_round,draft_position,draft_year,current_salary,hof_induction_year
0,1809,Robert Blackmon,DB,6-0,208.0,None,1967-05-12,"Bay City, TX",None,Baylor,"Van Vleck, TX",Seattle Seahawks,2.0,34.0,1990.0,None,NaN
1,23586,Dean Wells,LB,6-3,248.0,None,1970-07-20,"Louisville, KY",None,Kentucky,"Holy Cross, KY",Seattle Seahawks,4.0,85.0,1993.0,None,NaN
2,355,Kiko Alonso,ILB,6-3,238.0,Miami Dolphins,1990-08-14,"Newton, MA",None,Oregon,"Los Gatos, CA",Buffalo Bills,2.0,46.0,2013.0,"1,075,000",NaN
3,18182,Steve Ramsey,QB,6-2,210.0,None,1948-04-22,"Dallas, TX",1999-10-15,North Texas,"W.W. Samuell, TX",New Orleans Saints,5.0,126.0,1970.0,None,NaN
4,16250,Cory Nelms,CB,6-0,195.0,None,1988-02-27,"Neptune, NJ",None,Miami (FL),"Neptune, NJ",None,NaN,NaN,NaN,None,NaN


And, just for fun, show the player with the highest Current Salary from that dataset:

In [16]:
nfl_football_players.sort_values('current_salary', ascending=False).head(1)

,player_id,name,position,height,weight,current_team,birth_date,birth_place,death_date,college,high_school,draft_team,draft_round,draft_position,draft_year,current_salary,hof_induction_year
6454,721,Jeremiah Attaochu,OLB,6-3,252.0,Los Angeles Chargers,1993-01-17,"Ibadan, Nigeria",None,Georgia Tech,"Archbishop Carroll, DE",San Diego Chargers,2.0,50.0,2014.0,"993,150",NaN


## Fixing up the data
Assuming you did something like sort_values on one of the original columns, you probably got the wrong result.

Looking a bit more closely at the results, you'll notice that the current_salary column.  Remembering that we have made the shift from pythonic to pandorable, we can leverage the impressive-sounding "vectorized string functions" mentioned in Section 7.3 of the McKinney book.  Specifically, we can use the str.replace(...) method.  Note that had we use read_csv to load the file we could have used the ```thousands=``` option and avoided all this, but sometimes data doesn't come in a convenient format.

One way to apply functions is to operate on a column and then assign the results to another column.  For example, if we wanted to eliminate commas, we could replace them with null strings

In [17]:
nfl_football_players['current_salary'].str.replace(',', '')

0           None
1           None
2        1075000
3           None
4           None
          ...   
25038       None
25039       None
25040       None
25041       None
25042       None
Name: current_salary, Length: 25043, dtype: object

And assign the results to a column in the original dataframe (in this case I'm calling the column current_salary_nocommas). We need to assign it to a column to ensure the operation is done in-place.

In [18]:
nfl_football_players['current_salary_nocommas'] = nfl_football_players['current_salary'].str.replace(',', '')

In [19]:
nfl_football_players['current_salary_nocommas'].dtype

dtype('O')

But you'll notice that the type of the column is string, and we want to convert it to a float so we can sort it numerically.  So we can use the astype() function to convert it:

In [20]:
nfl_football_players['current_salary_cleaned'] = nfl_football_players['current_salary_nocommas'].astype(float)

In [21]:
nfl_football_players.head(2)

,player_id,name,position,height,weight,current_team,birth_date,birth_place,death_date,college,high_school,draft_team,draft_round,draft_position,draft_year,current_salary,hof_induction_year,current_salary_nocommas,current_salary_cleaned
0,1809,Robert Blackmon,DB,6-0,208.0,None,1967-05-12,"Bay City, TX",None,Baylor,"Van Vleck, TX",Seattle Seahawks,2.0,34.0,1990.0,None,NaN,None,NaN
1,23586,Dean Wells,LB,6-3,248.0,None,1970-07-20,"Louisville, KY",None,Kentucky,"Holy Cross, KY",Seattle Seahawks,4.0,85.0,1993.0,None,NaN,None,NaN


And now we can re-run our command to sort by salary and get the correct result:

In [22]:
nfl_football_players.sort_values('current_salary_cleaned', ascending=False).head(1)

,player_id,name,position,height,weight,current_team,birth_date,birth_place,death_date,college,high_school,draft_team,draft_round,draft_position,draft_year,current_salary,hof_induction_year,current_salary_nocommas,current_salary_cleaned
17756,4644,Kirk Cousins,QB,6-3,214.0,Washington Redskins,1988-08-19,"Holland, MI",None,Michigan St.,"Holland Christian, MI",Washington Redskins,4.0,102.0,2012.0,"23,943,600",NaN,23943600,23943600.0


## Dropping missing values

In addition to the "all" or "any" functionality described in McKinney section 7.1, it's sometimes useful to drop a row only if a certain column or columns have missing data.  To do this, use the subset= option with dropna().  So, for example, to drop all players for whom we do not have salary information, we could use the following code:

In [123]:
nfl_football_players['current_salary_cleaned'].isna().sum()

23278

In [24]:
nfl_football_players_salaries = nfl_football_players.dropna(subset=['current_salary_cleaned'])

In [25]:
nfl_football_players_salaries.head()

,player_id,name,position,height,weight,current_team,birth_date,birth_place,death_date,college,high_school,draft_team,draft_round,draft_position,draft_year,current_salary,hof_induction_year,current_salary_nocommas,current_salary_cleaned
2,355,Kiko Alonso,ILB,6-3,238.0,Miami Dolphins,1990-08-14,"Newton, MA",None,Oregon,"Los Gatos, CA",Buffalo Bills,2.0,46.0,2013.0,"1,075,000",NaN,1075000,1075000.0
6,2701,Preston Brown,ILB,6-1,251.0,Buffalo Bills,1992-10-27,"Cincinnati, OH",None,Louisville,"Northwest, OH",Buffalo Bills,3.0,73.0,2014.0,"1,762,000",NaN,1762000,1762000.0
13,3966,Frank Clark,DE,6-2,270.0,Seattle Seahawks,1993-06-14,"Cleveland, OH",None,Michigan,"Glenville, OH",Seattle Seahawks,2.0,63.0,2015.0,"774,294",NaN,774294,774294.0
37,19449,Emmanuel Sanders,WR,5-11,186.0,Denver Broncos,1987-03-16,"Bellville, TX",None,SMU,"Bellville, TX",Pittsburgh Steelers,3.0,82.0,2010.0,"6,750,000",NaN,6750000,6750000.0
53,17858,Bilal Powell,RB,5-11,204.0,New York Jets,1988-10-27,"Lakeland, FL",None,Louisville,"Lake Gibson, FL",New York Jets,4.0,126.0,2011.0,"3,750,000",NaN,3750000,3750000.0


## Creating dummy variables

We might, on occasion, want to "bin" or "discretize" a variable.  For example, we might want to take the previous dataframe and add dummy variables that map onto whether the salaries are "small" (< \\$1M) , "medium" (\\$1M - \\$10M), or "large" (> \\$10M). A **dummy variable** is a binary indicator variable (often used in regressions) to represent a categorical variable. 

We could do something like the following (I encourage you to make detailed notes about the following lines; we do several important steps.)

In [26]:
bins = [0,1000000,10000000,1000000000]

In [27]:
pd.cut(nfl_football_players_salaries['current_salary_cleaned'],bins,labels=['small','medium','large'])[0:10]

2      medium
6      medium
13      small
37     medium
53     medium
57      small
90      small
106    medium
113     small
114    medium
Name: current_salary_cleaned, dtype: category
Categories (3, object): [small < medium < large]

In [28]:
dummies = pd.get_dummies(pd.cut(nfl_football_players_salaries['current_salary_cleaned'],bins,labels=['small','medium','large']))

In [29]:
dummies.head()

current_salary_cleaned,small,medium,large
2,0,1,0
6,0,1,0
13,1,0,0
37,0,1,0
53,0,1,0


pd.concat stacks together objects along an axis _(Section 8.2 McKinney)_

In [30]:
nfl_cats = pd.concat([nfl_football_players_salaries,dummies],axis=1)

In [31]:
nfl_cats.tail()

,player_id,name,position,height,weight,current_team,birth_date,birth_place,death_date,college,...,draft_round,draft_position,draft_year,current_salary,hof_induction_year,current_salary_nocommas,current_salary_cleaned,small,medium,large
24885,13753,Paxton Lynch,QB,6-7,244.0,Denver Broncos,1994-02-12,"San Antonio, TX",None,Memphis,...,1.0,26.0,2016.0,"880,741",NaN,880741,880741.0,1,0,0
24917,22916,Jason Verrett,CB,5-10,189.0,Los Angeles Chargers,1991-05-13,"Fairfield, CA",None,TCU,...,1.0,25.0,2014.0,"860,000",NaN,860000,860000.0,1,0,0
24923,18586,Jordan Richards,SS,5-11,211.0,New England Patriots,1993-01-21,"Sacramento, CA",None,Stanford,...,2.0,64.0,2015.0,"772,413",NaN,772413,772413.0,1,0,0
24967,7063,Michael Floyd,WR,6-3,220.0,Minnesota Vikings,1989-11-27,"St. Paul, MN",None,Notre Dame,...,1.0,13.0,2012.0,"887,058",NaN,887058,887058.0,1,0,0
24984,10701,Chris Ivory,RB,6-0,222.0,Jacksonville Jaguars,1988-03-22,"Longview, TX",None,Washington St.,...,NaN,NaN,NaN,"4,500,000",NaN,4500000,4500000.0,0,1,0


## <font color="magenta">Q2 (12 points, 10 minutes): 
Create a new dataframe that contains all the columns in the nfl_football_players dataframe as well as an additional column that contains each player's height in centimeters. Show the first 5 rows of your result. </font>

**Hints:** 
- 1 inch = 2.54 cm
- you can use the vectorized string function str.split() to separate feet and inches from the original dataframe column
- remember to cast strings to numeric types if you're going to perform math on them
- you might want to create an intermediate (temporary) DataFrame to help you keep things clear instead of attempting to do 
this in one line

In [32]:
height_table=nfl_football_players.copy()
height_table['height'] = height_table['height'].str.replace('-', '.')
height_table['height'] = height_table['height'].astype(float)
height_table['height']=height_table['height']*2.54*12
height_table.head()

,player_id,name,position,height,weight,current_team,birth_date,birth_place,death_date,college,high_school,draft_team,draft_round,draft_position,draft_year,current_salary,hof_induction_year,current_salary_nocommas,current_salary_cleaned
0,1809,Robert Blackmon,DB,182.880,208.0,None,1967-05-12,"Bay City, TX",None,Baylor,"Van Vleck, TX",Seattle Seahawks,2.0,34.0,1990.0,None,NaN,None,NaN
1,23586,Dean Wells,LB,192.024,248.0,None,1970-07-20,"Louisville, KY",None,Kentucky,"Holy Cross, KY",Seattle Seahawks,4.0,85.0,1993.0,None,NaN,None,NaN
2,355,Kiko Alonso,ILB,192.024,238.0,Miami Dolphins,1990-08-14,"Newton, MA",None,Oregon,"Los Gatos, CA",Buffalo Bills,2.0,46.0,2013.0,"1,075,000",NaN,1075000,1075000.0
3,18182,Steve Ramsey,QB,188.976,210.0,None,1948-04-22,"Dallas, TX",1999-10-15,North Texas,"W.W. Samuell, TX",New Orleans Saints,5.0,126.0,1970.0,None,NaN,None,NaN
4,16250,Cory Nelms,CB,182.880,195.0,None,1988-02-27,"Neptune, NJ",None,Miami (FL),"Neptune, NJ",None,NaN,NaN,NaN,None,NaN,None,NaN


In [33]:
nfl_football_players_salaries_split = nfl_football_players_salaries.height.str.split('-')
nfl_football_players_salaries['feet_inches'] = nfl_football_players_salaries_split.apply(lambda x: x[0]).apply(lambda x: int(x)*12)
nfl_football_players_salaries['inches_inches'] = nfl_football_players_salaries_split.apply(lambda x: x[1]).apply(lambda x: int(x))
nfl_football_players_salaries['cm'] = (nfl_football_players_salaries['feet_inches'] + nfl_football_players_salaries['inches_inches'])*2.54
nfl_football_players_salaries.head()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = val

,player_id,name,position,height,weight,current_team,birth_date,birth_place,death_date,college,...,draft_round,draft_position,draft_year,current_salary,hof_induction_year,current_salary_nocommas,current_salary_cleaned,feet_inches,inches_inches,cm
2,355,Kiko Alonso,ILB,6-3,238.0,Miami Dolphins,1990-08-14,"Newton, MA",None,Oregon,...,2.0,46.0,2013.0,"1,075,000",NaN,1075000,1075000.0,72,3,190.50
6,2701,Preston Brown,ILB,6-1,251.0,Buffalo Bills,1992-10-27,"Cincinnati, OH",None,Louisville,...,3.0,73.0,2014.0,"1,762,000",NaN,1762000,1762000.0,72,1,185.42
13,3966,Frank Clark,DE,6-2,270.0,Seattle Seahawks,1993-06-14,"Cleveland, OH",None,Michigan,...,2.0,63.0,2015.0,"774,294",NaN,774294,774294.0,72,2,187.96
37,19449,Emmanuel Sanders,WR,5-11,186.0,Denver Broncos,1987-03-16,"Bellville, TX",None,SMU,...,3.0,82.0,2010.0,"6,750,000",NaN,6750000,6750000.0,60,11,180.34
53,17858,Bilal Powell,RB,5-11,204.0,New York Jets,1988-10-27,"Lakeland, FL",None,Louisville,...,4.0,126.0,2011.0,"3,750,000",NaN,3750000,3750000.0,60,11,180.34


## Scraping Tables from HTML

The ```pd.read_html``` function returns a list of DataFrames read from an HTML source.  The following line will return a list of DataFrames from https://en.wikipedia.org/wiki/List_of_largest_sports_contracts

In [34]:
contracts_scraped = pd.read_html('https://en.wikipedia.org/wiki/List_of_largest_sports_contracts',header=0)

In [35]:
len(contracts_scraped)

1

To get the first table, you'll need to pull off the 0th element:

In [36]:
contracts = contracts_scraped[0]
contracts.head()

,Rank,Player,Team/Streaming Service,Sport,Length of contract,Contract value (USD),Average per year (USD),Average per fixture (USD),Ref
0,01,Mike Trout,Los Angeles Angels,Baseball,12 years (2019–2030),"$430,000,000","$35,833,333","$219,393.42",[1]
1,02,Canelo Álvarez,DAZN*,Boxing,5 years (2018–2023),"$365,000,000","$73,000,000","$33,181,818.18",[2]
2,03,Bryce Harper,Philadelphia Phillies,Baseball,13 years (2019–2031),"$330,000,000","$25,384,615","$156,695.16",[3]
3,04,Giancarlo Stanton,Miami Marlins*,Baseball,13 years (2015–2027),"$325,000,000","$25,000,000","$154,320.99",[4]
4,05,Gerrit Cole,New York Yankees,Baseball,9 years (2020-2028),"$324,000,000","$36,000,000",NaN,NaN


## <font color="magenta"> Q3 (2 points): 
Count the number of players from each sport in the List of Largest Sports Contracts. 
Hint:  see value_counts() description above
<font>

In [37]:
contracts.Sport.value_counts()

Baseball                59
Basketball              24
American football       15
Auto racing              2
Ice hockey               1
Boxing                   1
Association football     1
Name: Sport, dtype: int64

Most players are baseball players, I can sort of assume baseball might be most popular sports during that time, and surprisingly, American football is not that popular as I thought.

# Joining, Combining, and Reshaping

In [38]:
import pandas as pd

In [39]:
frodo_url = 'https://en.wikipedia.org/wiki/Frodo_Baggins'
frodo_tables = pd.read_html(frodo_url)
frodo_tables[0]

,Frodo Baggins,Frodo Baggins.1
0,Tolkien character,Tolkien character
1,First appearance,The Fellowship of the Ring (1954)
2,Last appearance,Bilbo's Last Song (1974)
3,Portrayed by,Elijah Wood Kiran Shah (scale double)
4,Information,Information
5,Aliases,"Mr. Underhill,Maura Labingi"
6,Race,Hobbit
7,Book(s),The Fellowship of the RingThe Two TowersThe Re...


Now let's load the page for [Legolas](https://en.wikipedia.org/wiki/Legolas):

In [40]:
legolas_url = 'https://en.wikipedia.org/wiki/Legolas'
legolas_tables = pd.read_html(legolas_url)
legolas_tables[0]

,0,1
0,NaN,This article relies too much on references to ...


Hmmmm.  That doesn't look quite right.

Let's take a look at some URLs and figure out what's going on:
https://en.wikipedia.org/wiki/Legolas

### Inspect the Frodo and Legolas pages and see if you can figure out some _attributes_ of the table we're interested in.


There are more than one tables on this page, and different talbes are in different classes. The table we are looking for is belong 'class infobox'.

You'll notice that there are some characteristics that the "Information" box share across pages.  We can leverage that 
information by using the ```attrs``` attribute of ```read_html```.  For example, if we wanted to extract  the element(s) that had
an ```id``` of ```info```, we could use

```pd.read_html(url,{'id':'info'})```



## <font color="magenta">Q4: (2 points) Fill in the following code block to extract only the "Information" table for the Legolas page:

In [41]:
a = {'class': 'infobox'} # create an appropriate dictionary
pd.read_html(legolas_url, attrs=a)

[             Legolas                                    Legolas.1
 0  Tolkien character                            Tolkien character
 1        Information                                  Information
 2            Aliases  Greenleaf, (Legolas translatedinto English)
 3               Race                                   Sindar Elf
 4             Gender                                         Male
 5               Home                               Woodland realm
 6            Book(s)                        The Lord of the Rings]

Now let's define a function that, given a Wikipedia URL, will extract the contents of the Aliases component of the infobox table:

In [42]:
def get_aliases(url):
    tables = pd.read_html(url, attrs={'class':'infobox'}) # extract only tables with class=infobox
    print(url,len(tables))   # sanity check: we should have just 1 table
    infotable = tables[0]    # pull the first table into a DataFrame
    ret = ''                 # initialize an empty string for our return value
    try:                     # in case the next line throws an exception
        x = infotable.set_index(infotable.columns[0]).loc['Aliases'] # setting the index on column 0 will allow us to use .loc to look up the value of 'Aliases'
        ret = x.values[0]
    except:
        ret = 'None'
    return ret

And let's try it out:

In [43]:
get_aliases(legolas_url)

https://en.wikipedia.org/wiki/Legolas 1


'Greenleaf, (Legolas translatedinto English)'

So far, so good.  It seems to work.  Now let's set up a DataFrame with a bunch of URLs:

In [44]:
urls = ['https://en.wikipedia.org/wiki/Gimli_(Middle-earth)',
        'https://en.wikipedia.org/wiki/Frodo_Baggins',
        'https://en.wikipedia.org/wiki/Legolas',
        'https://en.wikipedia.org/wiki/Bilbo_Baggins',
        'https://en.wikipedia.org/wiki/Samwise_Gamgee',
        'https://en.wikipedia.org/wiki/Peregrin_Took',
        'https://en.wikipedia.org/wiki/Boromir',
        'https://en.wikipedia.org/wiki/Galadriel',
        'https://en.wikipedia.org/wiki/Meriadoc_Brandybuck']
names = ['Gimli',
         'Frodo',
         'Legolas',
         'Bilbo',
         'Sam',
         'Pippin',
         'Boromir',
         'Galadriel',
         'Meriadoc']

In [45]:
url_df = pd.DataFrame()
url_df['name'] = names
url_df['url'] = urls

In [46]:
url_df

,name,url
0,Gimli,https://en.wikipedia.org/wiki/Gimli_(Middle-ea...
1,Frodo,https://en.wikipedia.org/wiki/Frodo_Baggins
2,Legolas,https://en.wikipedia.org/wiki/Legolas
3,Bilbo,https://en.wikipedia.org/wiki/Bilbo_Baggins
4,Sam,https://en.wikipedia.org/wiki/Samwise_Gamgee
5,Pippin,https://en.wikipedia.org/wiki/Peregrin_Took
6,Boromir,https://en.wikipedia.org/wiki/Boromir
7,Galadriel,https://en.wikipedia.org/wiki/Galadriel
8,Meriadoc,https://en.wikipedia.org/wiki/Meriadoc_Brandybuck


The pythonic way of iterating through each of those rows would involve the use of some sort of ```for``` loop.  In pandas,
however, you can use the ```apply``` function to process an entire column!

In [47]:
url_df['url'].apply(get_aliases)

https://en.wikipedia.org/wiki/Gimli_(Middle-earth) 1
https://en.wikipedia.org/wiki/Frodo_Baggins 1
https://en.wikipedia.org/wiki/Legolas 1
https://en.wikipedia.org/wiki/Bilbo_Baggins 1
https://en.wikipedia.org/wiki/Samwise_Gamgee 1
https://en.wikipedia.org/wiki/Peregrin_Took 1
https://en.wikipedia.org/wiki/Boromir 1
https://en.wikipedia.org/wiki/Galadriel 1
https://en.wikipedia.org/wiki/Meriadoc_Brandybuck 1


0    Elf-friend Lockbearer Lord of the Glittering C...
1                          Mr. Underhill,Maura Labingi
2          Greenleaf, (Legolas translatedinto English)
3                                        Bilba Labingi
4    Samwise Gardner, Sam, Samwise the Brave,Banazî...
5    Pippin, Pip,"Ernil i Pheriannath"Thain Peregri...
6    Captain of the White Tower,High Warden of the ...
7                       AlatárielAltárielArtanisNerwen
8    Merry,Kalimac Brandagamba,Meriadoc the Magnifi...
Name: url, dtype: object

We can take the resulting Series and assign it to a new column in our DataFrame:

In [48]:
url_df['aliases'] = url_df['url'].apply(get_aliases)

https://en.wikipedia.org/wiki/Gimli_(Middle-earth) 1
https://en.wikipedia.org/wiki/Frodo_Baggins 1
https://en.wikipedia.org/wiki/Legolas 1
https://en.wikipedia.org/wiki/Bilbo_Baggins 1
https://en.wikipedia.org/wiki/Samwise_Gamgee 1
https://en.wikipedia.org/wiki/Peregrin_Took 1
https://en.wikipedia.org/wiki/Boromir 1
https://en.wikipedia.org/wiki/Galadriel 1
https://en.wikipedia.org/wiki/Meriadoc_Brandybuck 1


In [49]:
url_df

,name,url,aliases
0,Gimli,https://en.wikipedia.org/wiki/Gimli_(Middle-ea...,Elf-friend Lockbearer Lord of the Glittering C...
1,Frodo,https://en.wikipedia.org/wiki/Frodo_Baggins,"Mr. Underhill,Maura Labingi"
2,Legolas,https://en.wikipedia.org/wiki/Legolas,"Greenleaf, (Legolas translatedinto English)"
3,Bilbo,https://en.wikipedia.org/wiki/Bilbo_Baggins,Bilba Labingi
4,Sam,https://en.wikipedia.org/wiki/Samwise_Gamgee,"Samwise Gardner, Sam, Samwise the Brave,Banazî..."
5,Pippin,https://en.wikipedia.org/wiki/Peregrin_Took,"Pippin, Pip,""Ernil i Pheriannath""Thain Peregri..."
6,Boromir,https://en.wikipedia.org/wiki/Boromir,"Captain of the White Tower,High Warden of the ..."
7,Galadriel,https://en.wikipedia.org/wiki/Galadriel,AlatárielAltárielArtanisNerwen
8,Meriadoc,https://en.wikipedia.org/wiki/Meriadoc_Brandybuck,"Merry,Kalimac Brandagamba,Meriadoc the Magnifi..."


Let's just put the ```url_df``` DataFrame aside for now.  We'll return to it later.

## Creating DataFrames and Exploring Indexes

Let's load the usual libraries...

In [50]:
import pandas as pd
import numpy as np

Let's create some lists of data that we can use to construct a DataFrame:

In [51]:
names = ['Gandalf',
         'Gimli',
         'Frodo',
         'Legolas',
         'Bilbo',
         'Sam',
         'Pippin',
         'Boromir',
         'Aragorn',
         'Galadriel',
         'Meriadoc',
        'Lily']
races = ['Maia',
         'Dwarf',
         'Hobbit',
         'Elf',
         'Hobbit',
         'Hobbit',
         'Hobbit',
         'Man',
         'Man',
         'Elf',
         'Hobbit',
        'Hobbit']
magic = [10, 1, 4, 6, 4, 2, 0, 0, 2, 9, 0, np.NaN]
aggression = [7, 10, 2, 5, 1, 6, 3, 8, 7, 2, 4, np.NaN ]
stealth = [8, 2, 5, 10, 5, 4 ,5, 3, 9, 10, 6, np.NaN]

There are a few different ways to construct a DataFrame.  We can either use an empty constructor and assign Series:

## <font color="magenta"> Q5: (2 points) Construct a dataframe with 5 columns (names, races, magic, aggression, and stealth) using the lists above.

In [52]:
df = pd.DataFrame()
df['names'] = names
df['races'] = races
df['majic'] = magic
df['aggression'] = aggression
df['stealth'] = stealth 

In [53]:
df

,names,races,majic,aggression,stealth
0,Gandalf,Maia,10.0,7.0,8.0
1,Gimli,Dwarf,1.0,10.0,2.0
2,Frodo,Hobbit,4.0,2.0,5.0
3,Legolas,Elf,6.0,5.0,10.0
4,Bilbo,Hobbit,4.0,1.0,5.0
5,Sam,Hobbit,2.0,6.0,4.0
6,Pippin,Hobbit,0.0,3.0,5.0
7,Boromir,Man,0.0,8.0,3.0
8,Aragorn,Man,2.0,7.0,9.0
9,Galadriel,Elf,9.0,2.0,10.0


Alternatively, we could have set things up with a dict:

In [54]:
df = pd.DataFrame({'name': names,'race':races,'magic':magic,'aggression': aggression,'stealth':stealth})

In [55]:
df.head()

,name,race,magic,aggression,stealth
0,Gandalf,Maia,10.0,7.0,8.0
1,Gimli,Dwarf,1.0,10.0,2.0
2,Frodo,Hobbit,4.0,2.0,5.0
3,Legolas,Elf,6.0,5.0,10.0
4,Bilbo,Hobbit,4.0,1.0,5.0


Let's take a look at the index on the resulting DataFrame:

In [56]:
df.index

RangeIndex(start=0, stop=12, step=1)

We can set the index to something more useful than the default RangeIndex:

In [57]:
df_nameindexed = df.set_index('name')

And if we take a look at the results, we see that we have a pandas Index instead of a RangeIndex:

In [58]:
df_nameindexed.index

Index(['Gandalf', 'Gimli', 'Frodo', 'Legolas', 'Bilbo', 'Sam', 'Pippin',
       'Boromir', 'Aragorn', 'Galadriel', 'Meriadoc', 'Lily'],
      dtype='object', name='name')

In [59]:
df_nameindexed

,race,magic,aggression,stealth
name,,,,
Gandalf,Maia,10.0,7.0,8.0
Gimli,Dwarf,1.0,10.0,2.0
Frodo,Hobbit,4.0,2.0,5.0
Legolas,Elf,6.0,5.0,10.0
Bilbo,Hobbit,4.0,1.0,5.0
Sam,Hobbit,2.0,6.0,4.0
Pippin,Hobbit,0.0,3.0,5.0
Boromir,Man,0.0,8.0,3.0
Aragorn,Man,2.0,7.0,9.0


Setting the name Series as the index allows us to do things like:

In [60]:
df_nameindexed.loc['Aragorn']

race          Man
magic           2
aggression      7
stealth         9
Name: Aragorn, dtype: object

Now recall the Hierarchical indexing from the readings. _(Section 8.1 McKinney)_ We can pass a list of column names to set_index to create a Hierarchical Index:

In [61]:
df_racename_indexed = df.set_index(['race','name'])

In [62]:
df_racename_indexed.index

MultiIndex([(  'Maia',   'Gandalf'),
            ( 'Dwarf',     'Gimli'),
            ('Hobbit',     'Frodo'),
            (   'Elf',   'Legolas'),
            ('Hobbit',     'Bilbo'),
            ('Hobbit',       'Sam'),
            ('Hobbit',    'Pippin'),
            (   'Man',   'Boromir'),
            (   'Man',   'Aragorn'),
            (   'Elf', 'Galadriel'),
            ('Hobbit',  'Meriadoc'),
            ('Hobbit',      'Lily')],
           names=['race', 'name'])

This will allow us to get a DataFrame that matches a value on the outer index:

In [63]:
df_racename_indexed.loc['Hobbit']

,magic,aggression,stealth
name,,,
Frodo,4.0,2.0,5.0
Bilbo,4.0,1.0,5.0
Sam,2.0,6.0,4.0
Pippin,0.0,3.0,5.0
Meriadoc,0.0,4.0,6.0
Lily,NaN,NaN,NaN


We can also use the index on a Series to match the outer index:

In [64]:
df_racename_indexed['magic'].loc['Hobbit']


name
Frodo       4.0
Bilbo       4.0
Sam         2.0
Pippin      0.0
Meriadoc    0.0
Lily        NaN
Name: magic, dtype: float64

Or both indexes:

In [65]:
df_racename_indexed['magic'].loc['Hobbit','Frodo']

4.0

Or just the inner index:

In [66]:
df_racename_indexed['magic'].loc[:,'Frodo']

race
Hobbit    4.0
Name: magic, dtype: float64

## <font color="magenta"> Q6: (2 points) Using .loc find how much aggression Legalos, an Elf, has.

In [67]:
df_racename_indexed['aggression'].loc['Elf','Legolas']

5.0

## Stacking and Unstacking

Stacking takes "wide" data and makes it "taller"

In [68]:
df.set_index(['race']).stack()

race              
Maia    name            Gandalf
        magic                10
        aggression            7
        stealth               8
Dwarf   name              Gimli
        magic                 1
        aggression           10
        stealth               2
Hobbit  name              Frodo
        magic                 4
        aggression            2
        stealth               5
Elf     name            Legolas
        magic                 6
        aggression            5
        stealth              10
Hobbit  name              Bilbo
        magic                 4
        aggression            1
        stealth               5
        name                Sam
        magic                 2
        aggression            6
        stealth               4
        name             Pippin
        magic                 0
        aggression            3
        stealth               5
Man     name            Boromir
        magic                 0
        aggression   

If we call reset_index on the resulting Series, we get the following DataFrame:

In [69]:
df.set_index(['race']).stack().reset_index()

,race,level_1,0
0,Maia,name,Gandalf
1,Maia,magic,10
2,Maia,aggression,7
3,Maia,stealth,8
4,Dwarf,name,Gimli
5,Dwarf,magic,1
6,Dwarf,aggression,10
7,Dwarf,stealth,2
8,Hobbit,name,Frodo
9,Hobbit,magic,4


The column names in the above DataFrame aren't particularly helpful, so we can rename them:

In [70]:
df.set_index(['race']).stack().reset_index().rename(columns = {'level_0':'ID','level_1':'variable',0:'value'})

,race,variable,value
0,Maia,name,Gandalf
1,Maia,magic,10
2,Maia,aggression,7
3,Maia,stealth,8
4,Dwarf,name,Gimli
5,Dwarf,magic,1
6,Dwarf,aggression,10
7,Dwarf,stealth,2
8,Hobbit,name,Frodo
9,Hobbit,magic,4


You can do the opposite of stacking by using the ```unstack``` function:

In [71]:
df_stacked = df.stack()

In [72]:
df_stacked

0   name            Gandalf
    race               Maia
    magic                10
    aggression            7
    stealth               8
1   name              Gimli
    race              Dwarf
    magic                 1
    aggression           10
    stealth               2
2   name              Frodo
    race             Hobbit
    magic                 4
    aggression            2
    stealth               5
3   name            Legolas
    race                Elf
    magic                 6
    aggression            5
    stealth              10
4   name              Bilbo
    race             Hobbit
    magic                 4
    aggression            1
    stealth               5
5   name                Sam
    race             Hobbit
    magic                 2
    aggression            6
    stealth               4
6   name             Pippin
    race             Hobbit
    magic                 0
    aggression            3
    stealth               5
7   name            

In [73]:
df_stacked.unstack()

,name,race,magic,aggression,stealth
0,Gandalf,Maia,10,7,8
1,Gimli,Dwarf,1,10,2
2,Frodo,Hobbit,4,2,5
3,Legolas,Elf,6,5,10
4,Bilbo,Hobbit,4,1,5
5,Sam,Hobbit,2,6,4
6,Pippin,Hobbit,0,3,5
7,Boromir,Man,0,8,3
8,Aragorn,Man,2,7,9
9,Galadriel,Elf,9,2,10


Why would we want to stack or unstack?  It depends on what sorts of analyses we want to do "downstream".  It's also the basis for pivoting, melting, and pivot tables, which we'll cover later.

## Joining Data



Let's say we have another CSV file that contains URLs to Wikipedia pages for some of the LOTR characters:

In [74]:
urls = pd.read_csv('data/lotr_wikipedia.csv')
urls

,Unnamed: 0,name,url
0,0,Gandalf,https://en.wikipedia.org/wiki/Gandalf
1,1,Gimli,https://en.wikipedia.org/wiki/Gimli_(Middle-ea...
2,2,Frodo,https://en.wikipedia.org/wiki/Frodo_Baggins
3,3,Legolas,https://en.wikipedia.org/wiki/Legolas
4,4,Bilbo,https://en.wikipedia.org/wiki/Bilbo_Baggins
5,5,Sam,https://en.wikipedia.org/wiki/Samwise_Gamgee
6,6,Pippin,https://en.wikipedia.org/wiki/Peregrin_Took
7,7,Boromir,https://en.wikipedia.org/wiki/Boromir
8,8,Aragorn,https://en.wikipedia.org/wiki/Aragorn
9,9,Galadriel,https://en.wikipedia.org/wiki/Galadriel


In [75]:
urls = pd.read_csv('data/lotr_wikipedia.csv', index_col=0)

In [76]:
urls

,name,url
0,Gandalf,https://en.wikipedia.org/wiki/Gandalf
1,Gimli,https://en.wikipedia.org/wiki/Gimli_(Middle-ea...
2,Frodo,https://en.wikipedia.org/wiki/Frodo_Baggins
3,Legolas,https://en.wikipedia.org/wiki/Legolas
4,Bilbo,https://en.wikipedia.org/wiki/Bilbo_Baggins
5,Sam,https://en.wikipedia.org/wiki/Samwise_Gamgee
6,Pippin,https://en.wikipedia.org/wiki/Peregrin_Took
7,Boromir,https://en.wikipedia.org/wiki/Boromir
8,Aragorn,https://en.wikipedia.org/wiki/Aragorn
9,Galadriel,https://en.wikipedia.org/wiki/Galadriel


Let's take a look at the original DataFrame:

In [77]:
df

,name,race,magic,aggression,stealth
0,Gandalf,Maia,10.0,7.0,8.0
1,Gimli,Dwarf,1.0,10.0,2.0
2,Frodo,Hobbit,4.0,2.0,5.0
3,Legolas,Elf,6.0,5.0,10.0
4,Bilbo,Hobbit,4.0,1.0,5.0
5,Sam,Hobbit,2.0,6.0,4.0
6,Pippin,Hobbit,0.0,3.0,5.0
7,Boromir,Man,0.0,8.0,3.0
8,Aragorn,Man,2.0,7.0,9.0
9,Galadriel,Elf,9.0,2.0,10.0


It looks like the rows are "aligned", so we can use the ```concat``` function to concatenate the two DataFrames.
Note that we specify the axis to be the columns.  The default is to concatenate by rows, which isn't what we want.

In [78]:
pd.concat([df,urls],axis="columns")

,name,race,magic,aggression,stealth,name,url
0,Gandalf,Maia,10.0,7.0,8.0,Gandalf,https://en.wikipedia.org/wiki/Gandalf
1,Gimli,Dwarf,1.0,10.0,2.0,Gimli,https://en.wikipedia.org/wiki/Gimli_(Middle-ea...
2,Frodo,Hobbit,4.0,2.0,5.0,Frodo,https://en.wikipedia.org/wiki/Frodo_Baggins
3,Legolas,Elf,6.0,5.0,10.0,Legolas,https://en.wikipedia.org/wiki/Legolas
4,Bilbo,Hobbit,4.0,1.0,5.0,Bilbo,https://en.wikipedia.org/wiki/Bilbo_Baggins
5,Sam,Hobbit,2.0,6.0,4.0,Sam,https://en.wikipedia.org/wiki/Samwise_Gamgee
6,Pippin,Hobbit,0.0,3.0,5.0,Pippin,https://en.wikipedia.org/wiki/Peregrin_Took
7,Boromir,Man,0.0,8.0,3.0,Boromir,https://en.wikipedia.org/wiki/Boromir
8,Aragorn,Man,2.0,7.0,9.0,Aragorn,https://en.wikipedia.org/wiki/Aragorn
9,Galadriel,Elf,9.0,2.0,10.0,Galadriel,https://en.wikipedia.org/wiki/Galadriel


That's great, and it's consistent with what we've used in previous classes.  But what happens if the 
rows in the two DataFrames don't match up?  Let's load another file that has a slightly different
sequence of rows:

## <font color="magenta"> Q7: (2 points) Construct a dataframe called ```urls_wrong_order``` with lotr_wikipedia_wrong_order.csv which is in the data folder and concat it with df.

In [79]:
urls_wrong_order=pd.read_csv('data/lotr_wikipedia_wrong_order.csv')

Take a closer look at the name and url columns.  Something's not quite right.

We can work around that by using the appropriate indexing and then using the SQL-like ```merge``` function.

In [80]:
pd.concat([df,urls_wrong_order],axis='columns') # it's the same as pd.concat([df,urls_wrong_order],axis=1)

,name,race,magic,aggression,stealth,name,url
0,Gandalf,Maia,10.0,7.0,8.0,Boromir,https://en.wikipedia.org/wiki/Boromir
1,Gimli,Dwarf,1.0,10.0,2.0,Aragorn,https://en.wikipedia.org/wiki/Aragorn
2,Frodo,Hobbit,4.0,2.0,5.0,Gandalf,https://en.wikipedia.org/wiki/Gandalf
3,Legolas,Elf,6.0,5.0,10.0,Meriadoc,https://en.wikipedia.org/wiki/Meriadoc_Brandybuck
4,Bilbo,Hobbit,4.0,1.0,5.0,Gimli,https://en.wikipedia.org/wiki/Gimli_(Middle-ea...
5,Sam,Hobbit,2.0,6.0,4.0,Legolas,https://en.wikipedia.org/wiki/Legolas
6,Pippin,Hobbit,0.0,3.0,5.0,Bilbo,https://en.wikipedia.org/wiki/Bilbo_Baggins
7,Boromir,Man,0.0,8.0,3.0,Pippin,https://en.wikipedia.org/wiki/Peregrin_Took
8,Aragorn,Man,2.0,7.0,9.0,Galadriel,https://en.wikipedia.org/wiki/Galadriel
9,Galadriel,Elf,9.0,2.0,10.0,Lily,NaN


In [81]:
df_names = df.set_index('name')

In [82]:
df_names

,race,magic,aggression,stealth
name,,,,
Gandalf,Maia,10.0,7.0,8.0
Gimli,Dwarf,1.0,10.0,2.0
Frodo,Hobbit,4.0,2.0,5.0
Legolas,Elf,6.0,5.0,10.0
Bilbo,Hobbit,4.0,1.0,5.0
Sam,Hobbit,2.0,6.0,4.0
Pippin,Hobbit,0.0,3.0,5.0
Boromir,Man,0.0,8.0,3.0
Aragorn,Man,2.0,7.0,9.0


In [83]:
urls_wrong_order_names = urls_wrong_order.set_index('name')

In [84]:
df_names.join(urls_wrong_order_names) # it will join correspoindingly by index column

,race,magic,aggression,stealth,url
name,,,,,
Gandalf,Maia,10.0,7.0,8.0,https://en.wikipedia.org/wiki/Gandalf
Gimli,Dwarf,1.0,10.0,2.0,https://en.wikipedia.org/wiki/Gimli_(Middle-ea...
Frodo,Hobbit,4.0,2.0,5.0,https://en.wikipedia.org/wiki/Frodo_Baggins
Legolas,Elf,6.0,5.0,10.0,https://en.wikipedia.org/wiki/Legolas
Bilbo,Hobbit,4.0,1.0,5.0,https://en.wikipedia.org/wiki/Bilbo_Baggins
Sam,Hobbit,2.0,6.0,4.0,https://en.wikipedia.org/wiki/Samwise_Gamgee
Pippin,Hobbit,0.0,3.0,5.0,https://en.wikipedia.org/wiki/Peregrin_Took
Boromir,Man,0.0,8.0,3.0,https://en.wikipedia.org/wiki/Boromir
Aragorn,Man,2.0,7.0,9.0,https://en.wikipedia.org/wiki/Aragorn


In [85]:
df.head()

,name,race,magic,aggression,stealth
0,Gandalf,Maia,10.0,7.0,8.0
1,Gimli,Dwarf,1.0,10.0,2.0
2,Frodo,Hobbit,4.0,2.0,5.0
3,Legolas,Elf,6.0,5.0,10.0
4,Bilbo,Hobbit,4.0,1.0,5.0


In [86]:
urls_wrong_order.head()

,name,url
0,Boromir,https://en.wikipedia.org/wiki/Boromir
1,Aragorn,https://en.wikipedia.org/wiki/Aragorn
2,Gandalf,https://en.wikipedia.org/wiki/Gandalf
3,Meriadoc,https://en.wikipedia.org/wiki/Meriadoc_Brandybuck
4,Gimli,https://en.wikipedia.org/wiki/Gimli_(Middle-ea...


In [87]:
urls_wrong_order['name']

0       Boromir
1       Aragorn
2       Gandalf
3      Meriadoc
4         Gimli
5       Legolas
6         Bilbo
7        Pippin
8     Galadriel
9          Lily
10        Frodo
11          Sam
Name: name, dtype: object

In [88]:
df['name']

0       Gandalf
1         Gimli
2         Frodo
3       Legolas
4         Bilbo
5           Sam
6        Pippin
7       Boromir
8       Aragorn
9     Galadriel
10     Meriadoc
11         Lily
Name: name, dtype: object

In [89]:
df.merge(urls_wrong_order,on='name') # the same as when I set index as 'name', there are 'left_on', "right_on", refer to sections below.

,name,race,magic,aggression,stealth,url
0,Gandalf,Maia,10.0,7.0,8.0,https://en.wikipedia.org/wiki/Gandalf
1,Gimli,Dwarf,1.0,10.0,2.0,https://en.wikipedia.org/wiki/Gimli_(Middle-ea...
2,Frodo,Hobbit,4.0,2.0,5.0,https://en.wikipedia.org/wiki/Frodo_Baggins
3,Legolas,Elf,6.0,5.0,10.0,https://en.wikipedia.org/wiki/Legolas
4,Bilbo,Hobbit,4.0,1.0,5.0,https://en.wikipedia.org/wiki/Bilbo_Baggins
5,Sam,Hobbit,2.0,6.0,4.0,https://en.wikipedia.org/wiki/Samwise_Gamgee
6,Pippin,Hobbit,0.0,3.0,5.0,https://en.wikipedia.org/wiki/Peregrin_Took
7,Boromir,Man,0.0,8.0,3.0,https://en.wikipedia.org/wiki/Boromir
8,Aragorn,Man,2.0,7.0,9.0,https://en.wikipedia.org/wiki/Aragorn
9,Galadriel,Elf,9.0,2.0,10.0,https://en.wikipedia.org/wiki/Galadriel


Now let's add a few additional URLs:

In [90]:
urls_extras = pd.read_csv("data/lotr_wikipedia_extras.csv")

In [91]:
urls_extras

,name,url
0,Treebeard,https://en.wikipedia.org/wiki/Treebeard
1,Elrond,https://en.wikipedia.org/wiki/Elrond


And now let's use concat to add the new entries to the DataFrame.

In [92]:
urls_complete = pd.concat([urls,urls_extras]) # no need axis value, maybe because the two data frame structures are same

In [93]:
urls_complete

,name,url
0,Gandalf,https://en.wikipedia.org/wiki/Gandalf
1,Gimli,https://en.wikipedia.org/wiki/Gimli_(Middle-ea...
2,Frodo,https://en.wikipedia.org/wiki/Frodo_Baggins
3,Legolas,https://en.wikipedia.org/wiki/Legolas
4,Bilbo,https://en.wikipedia.org/wiki/Bilbo_Baggins
5,Sam,https://en.wikipedia.org/wiki/Samwise_Gamgee
6,Pippin,https://en.wikipedia.org/wiki/Peregrin_Took
7,Boromir,https://en.wikipedia.org/wiki/Boromir
8,Aragorn,https://en.wikipedia.org/wiki/Aragorn
9,Galadriel,https://en.wikipedia.org/wiki/Galadriel


Now that we've got a complete (for our purposes) list of URLs, let's use that DataFrame and our original
one to demonstrate the different types of ```join```s.

By default, ```join``` uses a left join, which means the all the values from the "left"
side are used, whether or not there's a corresponding entry from the "right" side.  In the example 
below, note that the url value for "Lily" is "NaN":

In [94]:
urls_wrong_order=urls_complete

In [95]:
df.merge(urls_wrong_order,on='name',how='left') # not working coz there's no corrosponding left index there

,name,race,magic,aggression,stealth,url
0,Gandalf,Maia,10.0,7.0,8.0,https://en.wikipedia.org/wiki/Gandalf
1,Gimli,Dwarf,1.0,10.0,2.0,https://en.wikipedia.org/wiki/Gimli_(Middle-ea...
2,Frodo,Hobbit,4.0,2.0,5.0,https://en.wikipedia.org/wiki/Frodo_Baggins
3,Legolas,Elf,6.0,5.0,10.0,https://en.wikipedia.org/wiki/Legolas
4,Bilbo,Hobbit,4.0,1.0,5.0,https://en.wikipedia.org/wiki/Bilbo_Baggins
5,Sam,Hobbit,2.0,6.0,4.0,https://en.wikipedia.org/wiki/Samwise_Gamgee
6,Pippin,Hobbit,0.0,3.0,5.0,https://en.wikipedia.org/wiki/Peregrin_Took
7,Boromir,Man,0.0,8.0,3.0,https://en.wikipedia.org/wiki/Boromir
8,Aragorn,Man,2.0,7.0,9.0,https://en.wikipedia.org/wiki/Aragorn
9,Galadriel,Elf,9.0,2.0,10.0,https://en.wikipedia.org/wiki/Galadriel


The "opposite" of a left join is, perhaps unsurprisingly, a "right" join, in which
all the values from the "right" side are used, whether or not a corresponding
value from the "left" side exists. Note in the following example that "Lily" has
disappeared, and Treebeard and Elrond lack information about "race", "magic", "aggression", and "stealth".

In [96]:
df.merge(urls_wrong_order,on='name',how='right') # tho the Treebeard and Elrond data shows, there are many NaN values

,name,race,magic,aggression,stealth,url
0,Gandalf,Maia,10.0,7.0,8.0,https://en.wikipedia.org/wiki/Gandalf
1,Gimli,Dwarf,1.0,10.0,2.0,https://en.wikipedia.org/wiki/Gimli_(Middle-ea...
2,Frodo,Hobbit,4.0,2.0,5.0,https://en.wikipedia.org/wiki/Frodo_Baggins
3,Legolas,Elf,6.0,5.0,10.0,https://en.wikipedia.org/wiki/Legolas
4,Bilbo,Hobbit,4.0,1.0,5.0,https://en.wikipedia.org/wiki/Bilbo_Baggins
5,Sam,Hobbit,2.0,6.0,4.0,https://en.wikipedia.org/wiki/Samwise_Gamgee
6,Pippin,Hobbit,0.0,3.0,5.0,https://en.wikipedia.org/wiki/Peregrin_Took
7,Boromir,Man,0.0,8.0,3.0,https://en.wikipedia.org/wiki/Boromir
8,Aragorn,Man,2.0,7.0,9.0,https://en.wikipedia.org/wiki/Aragorn
9,Galadriel,Elf,9.0,2.0,10.0,https://en.wikipedia.org/wiki/Galadriel


In addition to "left" and "right" joins, we have "outer" joins, which include
values from both the "left" and "right" DataFrames, regardless of whether
there are corresponding values in the other DataFrame.  Note that all of 
"Lily", "Treebeard" and "Elrond" are present in the following DataFrame:

In [97]:
df.merge(urls_wrong_order,on='name',how='outer')

,name,race,magic,aggression,stealth,url
0,Gandalf,Maia,10.0,7.0,8.0,https://en.wikipedia.org/wiki/Gandalf
1,Gimli,Dwarf,1.0,10.0,2.0,https://en.wikipedia.org/wiki/Gimli_(Middle-ea...
2,Frodo,Hobbit,4.0,2.0,5.0,https://en.wikipedia.org/wiki/Frodo_Baggins
3,Legolas,Elf,6.0,5.0,10.0,https://en.wikipedia.org/wiki/Legolas
4,Bilbo,Hobbit,4.0,1.0,5.0,https://en.wikipedia.org/wiki/Bilbo_Baggins
5,Sam,Hobbit,2.0,6.0,4.0,https://en.wikipedia.org/wiki/Samwise_Gamgee
6,Pippin,Hobbit,0.0,3.0,5.0,https://en.wikipedia.org/wiki/Peregrin_Took
7,Boromir,Man,0.0,8.0,3.0,https://en.wikipedia.org/wiki/Boromir
8,Aragorn,Man,2.0,7.0,9.0,https://en.wikipedia.org/wiki/Aragorn
9,Galadriel,Elf,9.0,2.0,10.0,https://en.wikipedia.org/wiki/Galadriel


Finally, there are "inner" joins, which include only those values that exist in both the "left" and "right" DataFrames:

In [98]:
df.merge(urls_wrong_order,on='name',how='inner') #only join the values that exsiting in both tables, so the url value for the 2 new rows are empyt.

,name,race,magic,aggression,stealth,url
0,Gandalf,Maia,10.0,7.0,8.0,https://en.wikipedia.org/wiki/Gandalf
1,Gimli,Dwarf,1.0,10.0,2.0,https://en.wikipedia.org/wiki/Gimli_(Middle-ea...
2,Frodo,Hobbit,4.0,2.0,5.0,https://en.wikipedia.org/wiki/Frodo_Baggins
3,Legolas,Elf,6.0,5.0,10.0,https://en.wikipedia.org/wiki/Legolas
4,Bilbo,Hobbit,4.0,1.0,5.0,https://en.wikipedia.org/wiki/Bilbo_Baggins
5,Sam,Hobbit,2.0,6.0,4.0,https://en.wikipedia.org/wiki/Samwise_Gamgee
6,Pippin,Hobbit,0.0,3.0,5.0,https://en.wikipedia.org/wiki/Peregrin_Took
7,Boromir,Man,0.0,8.0,3.0,https://en.wikipedia.org/wiki/Boromir
8,Aragorn,Man,2.0,7.0,9.0,https://en.wikipedia.org/wiki/Aragorn
9,Galadriel,Elf,9.0,2.0,10.0,https://en.wikipedia.org/wiki/Galadriel


Sometimes it's nice to know how a particular row got added to the resulting DataFrame.  Using ```indicator=True```
allows us to examine this:

In [99]:
df.merge(urls_wrong_order,how='outer',indicator=True)

,name,race,magic,aggression,stealth,url,_merge
0,Gandalf,Maia,10.0,7.0,8.0,https://en.wikipedia.org/wiki/Gandalf,both
1,Gimli,Dwarf,1.0,10.0,2.0,https://en.wikipedia.org/wiki/Gimli_(Middle-ea...,both
2,Frodo,Hobbit,4.0,2.0,5.0,https://en.wikipedia.org/wiki/Frodo_Baggins,both
3,Legolas,Elf,6.0,5.0,10.0,https://en.wikipedia.org/wiki/Legolas,both
4,Bilbo,Hobbit,4.0,1.0,5.0,https://en.wikipedia.org/wiki/Bilbo_Baggins,both
5,Sam,Hobbit,2.0,6.0,4.0,https://en.wikipedia.org/wiki/Samwise_Gamgee,both
6,Pippin,Hobbit,0.0,3.0,5.0,https://en.wikipedia.org/wiki/Peregrin_Took,both
7,Boromir,Man,0.0,8.0,3.0,https://en.wikipedia.org/wiki/Boromir,both
8,Aragorn,Man,2.0,7.0,9.0,https://en.wikipedia.org/wiki/Aragorn,both
9,Galadriel,Elf,9.0,2.0,10.0,https://en.wikipedia.org/wiki/Galadriel,both


You'll note that we used the ```merge``` function from the DataFrame and passed in the other DataFrame as an argument.
You can also call the ```merge``` function from pandas directly and pass it the two DataFrames you are merging:

![pivot 1](http://www.datasciencemadesimple.com/wp-content/uploads/2017/09/join-or-merge-in-python-pandas-1.png "pivots")

In [100]:
pd.merge(df,urls_wrong_order,how='outer',indicator=True)

,name,race,magic,aggression,stealth,url,_merge
0,Gandalf,Maia,10.0,7.0,8.0,https://en.wikipedia.org/wiki/Gandalf,both
1,Gimli,Dwarf,1.0,10.0,2.0,https://en.wikipedia.org/wiki/Gimli_(Middle-ea...,both
2,Frodo,Hobbit,4.0,2.0,5.0,https://en.wikipedia.org/wiki/Frodo_Baggins,both
3,Legolas,Elf,6.0,5.0,10.0,https://en.wikipedia.org/wiki/Legolas,both
4,Bilbo,Hobbit,4.0,1.0,5.0,https://en.wikipedia.org/wiki/Bilbo_Baggins,both
5,Sam,Hobbit,2.0,6.0,4.0,https://en.wikipedia.org/wiki/Samwise_Gamgee,both
6,Pippin,Hobbit,0.0,3.0,5.0,https://en.wikipedia.org/wiki/Peregrin_Took,both
7,Boromir,Man,0.0,8.0,3.0,https://en.wikipedia.org/wiki/Boromir,both
8,Aragorn,Man,2.0,7.0,9.0,https://en.wikipedia.org/wiki/Aragorn,both
9,Galadriel,Elf,9.0,2.0,10.0,https://en.wikipedia.org/wiki/Galadriel,both


## <font color="magenta">Q8: (2 points) Join the ```url_df``` DataFrame (that contains aliases) to the ```df``` DataFrame using an appropriate merge

In [101]:
url_df

,name,url,aliases
0,Gimli,https://en.wikipedia.org/wiki/Gimli_(Middle-ea...,Elf-friend Lockbearer Lord of the Glittering C...
1,Frodo,https://en.wikipedia.org/wiki/Frodo_Baggins,"Mr. Underhill,Maura Labingi"
2,Legolas,https://en.wikipedia.org/wiki/Legolas,"Greenleaf, (Legolas translatedinto English)"
3,Bilbo,https://en.wikipedia.org/wiki/Bilbo_Baggins,Bilba Labingi
4,Sam,https://en.wikipedia.org/wiki/Samwise_Gamgee,"Samwise Gardner, Sam, Samwise the Brave,Banazî..."
5,Pippin,https://en.wikipedia.org/wiki/Peregrin_Took,"Pippin, Pip,""Ernil i Pheriannath""Thain Peregri..."
6,Boromir,https://en.wikipedia.org/wiki/Boromir,"Captain of the White Tower,High Warden of the ..."
7,Galadriel,https://en.wikipedia.org/wiki/Galadriel,AlatárielAltárielArtanisNerwen
8,Meriadoc,https://en.wikipedia.org/wiki/Meriadoc_Brandybuck,"Merry,Kalimac Brandagamba,Meriadoc the Magnifi..."


In [102]:
df

,name,race,magic,aggression,stealth
0,Gandalf,Maia,10.0,7.0,8.0
1,Gimli,Dwarf,1.0,10.0,2.0
2,Frodo,Hobbit,4.0,2.0,5.0
3,Legolas,Elf,6.0,5.0,10.0
4,Bilbo,Hobbit,4.0,1.0,5.0
5,Sam,Hobbit,2.0,6.0,4.0
6,Pippin,Hobbit,0.0,3.0,5.0
7,Boromir,Man,0.0,8.0,3.0
8,Aragorn,Man,2.0,7.0,9.0
9,Galadriel,Elf,9.0,2.0,10.0


In [103]:
df.merge(url_df, how='left') # there's no right or wrong answer

,name,race,magic,aggression,stealth,url,aliases
0,Gandalf,Maia,10.0,7.0,8.0,NaN,NaN
1,Gimli,Dwarf,1.0,10.0,2.0,https://en.wikipedia.org/wiki/Gimli_(Middle-ea...,Elf-friend Lockbearer Lord of the Glittering C...
2,Frodo,Hobbit,4.0,2.0,5.0,https://en.wikipedia.org/wiki/Frodo_Baggins,"Mr. Underhill,Maura Labingi"
3,Legolas,Elf,6.0,5.0,10.0,https://en.wikipedia.org/wiki/Legolas,"Greenleaf, (Legolas translatedinto English)"
4,Bilbo,Hobbit,4.0,1.0,5.0,https://en.wikipedia.org/wiki/Bilbo_Baggins,Bilba Labingi
5,Sam,Hobbit,2.0,6.0,4.0,https://en.wikipedia.org/wiki/Samwise_Gamgee,"Samwise Gardner, Sam, Samwise the Brave,Banazî..."
6,Pippin,Hobbit,0.0,3.0,5.0,https://en.wikipedia.org/wiki/Peregrin_Took,"Pippin, Pip,""Ernil i Pheriannath""Thain Peregri..."
7,Boromir,Man,0.0,8.0,3.0,https://en.wikipedia.org/wiki/Boromir,"Captain of the White Tower,High Warden of the ..."
8,Aragorn,Man,2.0,7.0,9.0,NaN,NaN
9,Galadriel,Elf,9.0,2.0,10.0,https://en.wikipedia.org/wiki/Galadriel,AlatárielAltárielArtanisNerwen


# Aggregation and Grouping

In [104]:
import pandas as pd
import numpy as np

## Some more basic pandas functionality
One of the nice things about pandas is that it simplifies many common operations on datasets.  Let's load yet another LOTR dataset, 
this time using StringIO.  StringIO allows us to create a string that's then available as a file!

Why would you want to do this?  Sometimes it's easier to just paste data right into a Jupyter notebook (or python script)
than it is to create another CSV file.


In [105]:
from io import StringIO

LOTRDATA=StringIO("""name,race,gender,magic,aggression,stealth
Gandalf,Maia,Male,10.0,7.0,8.0
Gimli,Dwarf,None,1.0,10.0,2.0
Frodo,Hobbit,Male,4.0,2.0,5.0
Legolas,Elf,Male,6.0,5.0,10.0
Bilbo,Hobbit,Male,4.0,1.0,5.0
Sam,Hobbit,Male,2.0,6.0,4.0
Pippin,Hobbit,Male,0.0,3.0,5.0
Boromir,Human,Male,0.0,8.0,3.0
Aragorn,Human,Male,2.0,7.0,9.0
Galadriel,Elf,Female,9.0,2.0,10.0
Lily,Hobbit,Female,,,
Meriadoc,Hobbit,Male,,4.0,6.0
Melian,Maia,Female,10.0,5.0,9.0
Idril,Elf,Female,8.0,,8.0
""")

lotr = pd.read_csv(LOTRDATA, index_col=None)
lotr

,name,race,gender,magic,aggression,stealth
0,Gandalf,Maia,Male,10.0,7.0,8.0
1,Gimli,Dwarf,None,1.0,10.0,2.0
2,Frodo,Hobbit,Male,4.0,2.0,5.0
3,Legolas,Elf,Male,6.0,5.0,10.0
4,Bilbo,Hobbit,Male,4.0,1.0,5.0
5,Sam,Hobbit,Male,2.0,6.0,4.0
6,Pippin,Hobbit,Male,0.0,3.0,5.0
7,Boromir,Human,Male,0.0,8.0,3.0
8,Aragorn,Human,Male,2.0,7.0,9.0
9,Galadriel,Elf,Female,9.0,2.0,10.0


Now that we have a DataFrame, we can get some basic statistics about it using the ```describe()``` function.
Note that ```describe()``` only returns values for numeric columns.  Note too that it returns another DataFrame

In [106]:
lotr.describe() 
# lotr.describe().loc['mean'] I can just check mean values

,magic,aggression,stealth
count,12.000000,12.000000,13.000000
mean,4.666667,5.000000,6.461538
std,3.821788,2.730301,2.696151
min,0.000000,1.000000,2.000000
25%,1.750000,2.750000,5.000000
50%,4.000000,5.000000,6.000000
75%,8.250000,7.000000,9.000000
max,10.000000,10.000000,10.000000


In [107]:
type(lotr.describe())

pandas.core.frame.DataFrame

## Pivots and Pivot Tables (hightly recommended review)
The following cells are based on: 
http://nikgrozev.com/2015/07/01/reshaping-in-pandas-pivot-pivot-table-stack-and-unstack-explained-with-pictures/ , which is
one of the best guides to pivots, pivot tables, stacking and unstacking that I've encountered.

For demonstration purposes, let's create the same DataFrame that Nikolay Grozev uses in his tutorial:

In [108]:
from collections import OrderedDict
from pandas import DataFrame

table = OrderedDict((
    ("Item", ['Item0', 'Item0', 'Item1', 'Item1']),
    ('CType',['Gold', 'Bronze', 'Gold', 'Silver']),
    ('USD',  ['1$', '2$', '3$', '4$']),
    ('EU',   ['1€', '2€', '3€', '4€'])
))
metal = DataFrame(table)
metal

# create the table

,Item,CType,USD,EU
0,Item0,Gold,1$,1€
1,Item0,Bronze,2$,2€
2,Item1,Gold,3$,3€
3,Item1,Silver,4$,4€


In [109]:
# make a table of items (rows) and costs (USD) 
# for each in gold and bronze
metal.pivot(index='Item',columns='CType',values='USD')

CType,Bronze,Gold,Silver
Item,,,
Item0,2$,1$,NaN
Item1,NaN,3$,4$


See the image below... we're telling Pandas to take the table above, create a row for every item. 
This is done by setting index to Item (the column in the original table that contains item names)
We then are telling pandas we want to create a column for every unique element in the
original CType column.  And finally, we want the value in the cells to be the value from the USD
column in the original table.

![pivot 1](http://nikgrozev.com/images/blog/Reshaping%20in%20Pandas%20-%20Pivot%20Pivot-Table%20Stack%20and%20Unstack%20explained%20with%20Pictures/pivoting_simple1.png "pivots")

## <font color="magenta">Q9: (2 points) Using pivot and the dataframe below make a table with each person as the index, their purchased items as columns, and the price they paid as the values.</font>


In [110]:
table1 = OrderedDict((
    ("Person", ['Scott', 'Julie', 'Shiyan', 'Julie','Scott','Julie', 'Shiyan', 'Julie','Scott', 'Julie']),
    ('Item Purchased',['Kit-Kat','Mango', 'Twix', 'M&Ms', 'Ferrero Rocher','Apple', 'Watermelon', 'Pineapple', 'Snickers','Pear']),
    ('Price Paid',  ['$1','$2', '$1.5', '$2', '$1.5','$1','$1','$2', '$1.5', '$2']),
    ))
fav = DataFrame(table1)
fav

,Person,Item Purchased,Price Paid
0,Scott,Kit-Kat,$1
1,Julie,Mango,$2
2,Shiyan,Twix,$1.5
3,Julie,M&Ms,$2
4,Scott,Ferrero Rocher,$1.5
5,Julie,Apple,$1
6,Shiyan,Watermelon,$1
7,Julie,Pineapple,$2
8,Scott,Snickers,$1.5
9,Julie,Pear,$2


In [111]:
fav.pivot(index='Person', columns='Item Purchased', values='Price Paid')

Item Purchased,Apple,Ferrero Rocher,Kit-Kat,M&Ms,Mango,Pear,Pineapple,Snickers,Twix,Watermelon
Person,,,,,,,,,,
Julie,$1,NaN,NaN,$2,$2,$2,$2,NaN,NaN,NaN
Scott,NaN,$1.5,$1,NaN,NaN,NaN,NaN,$1.5,NaN,NaN
Shiyan,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,$1.5,$1


In [112]:
# end of exercise

In [113]:
p = metal.pivot(index='Item',columns='CType',values='USD')  # same as above, I'm just renaming the pivot as "p"

Access the USD cost of Item0 for Gold customers...

First we find the row for Item0/Gold and then we select the USD column and pull out the value

In [114]:
result = metal[((metal.Item == 'Item0') & (metal.CType == 'Gold'))].USD

In [115]:
type(result)

pandas.core.series.Series

In [116]:
result

0    1$
Name: USD, dtype: object

Do the same thing on pivoted table. Here we pull out the row for Item0, grab the Gold column and print the value

In [117]:
p[p.index == 'Item0'].Gold.values

array(['1$'], dtype=object)

Now pivot by multiple columns, I want USD and EU prices. It returns a hierarchical index.

In [118]:
metal.pivot(index='Item',columns='CType') #I dont need to specify the values attribute

USD                 EU            
CType Bronze Gold Silver Bronze Gold Silver
Item                                       
Item0     2$   1$    NaN     2€   1€    NaN
Item1    NaN   3$     4$    NaN   3€     4€

Access the USD cost of Item0 for Gold customers

In [119]:
p = metal.pivot(index='Item',columns='CType')
p.USD[p.USD.index == 'Item0'].Gold.values

array(['1$'], dtype=object)

# What happens if there is a collision? 
See the problem?  There are two Item0/Golds:


![pivot 2](http://nikgrozev.com/images/blog/Reshaping%20in%20Pandas%20-%20Pivot%20Pivot-Table%20Stack%20and%20Unstack%20explained%20with%20Pictures/pivoting_simple_error.png "pivots")

Let's set up another DataFrame to demonstrate this:

In [120]:
table = OrderedDict((
    ("Item", ['Item0', 'Item0', 'Item0', 'Item1']),
    ('CType',['Gold', 'Bronze', 'Gold', 'Silver']),
    ('USD',  ['1$', '2$', '3$', '4$']),
    ('EU',   ['1€', '2€', '3€', '4€'])
))
metal = DataFrame(table)


In [121]:
metal.head()

,Item,CType,USD,EU
0,Item0,Gold,1$,1€
1,Item0,Bronze,2$,2€
2,Item0,Gold,3$,3€
3,Item1,Silver,4$,4€


The next cell will generate an error:

In [122]:
p = metal.pivot(index='Item', columns='CType', values='USD')
# will return an error, since there's duplicate value

ValueError: Index contains duplicate entries, cannot reshape

## pivot_tables is your friend
![pivot 1](http://nikgrozev.com/images/blog/Reshaping%20in%20Pandas%20-%20Pivot%20Pivot-Table%20Stack%20and%20Unstack%20explained%20with%20Pictures/pivoting_table_simple1.png "pivots")

Let's create yet another DataFrame to play with:

In [ ]:
table = OrderedDict((
    ("Item", ['Item0', 'Item0', 'Item0', 'Item1']),
    ('CType',['Gold', 'Bronze', 'Gold', 'Silver']),
    ('USD',  [1, 2, 3, 4]),
    ('EU',   [1.1, 2.2, 3.3, 4.4])
))
metal = DataFrame(table)
metal

pivot_table is  a bit different than pivot... It's the same with the first part
index, columns, values remain the same as before BUT we added a rule (aggfunc)
that says: whey you hit a conflict, the way to resolve it is X (in this case
x is the "mean"... so find the mean of the two numbers)

In [ ]:
p = metal.pivot_table(index='Item',columns='CType',values='USD',aggfunc=np.mean)
p

You could have also resolved the conflict in other ways.  Here we tell it to take the "min":

In [ ]:
p = metal.pivot_table(index='Item',columns='CType',values='USD',aggfunc=np.min)
p

## <font color="magenta">Q10: (2 points) Using pivot_table and the dataframe below make a table with each person as the index, their purchased items as columns, and the sum of the price they paid as the values (sometimes people bought more than one of the same item).</font>


In [ ]:
table1 = OrderedDict((
    ("Person", ['Scott', 'Julie', 'Shiyan', 'Julie',
                'Scott','Julie', 'Shiyan', 'Julie','Scott', 
                'Julie','Scott', 'Julie', 'Shiyan', 'Julie',
                'Scott','Julie',]),
    ('Item Purchased',['Kit-Kat','Mango', 'Twix', 'M&Ms', 'Ferrero Rocher',
                       'Apple', 'Watermelon', 'Pineapple', 'Snickers','Pear','Kit-Kat','Mango', 'Twix', 'M&Ms', 'Ferrero Rocher',
                       'Apple']),
    ('Price Paid (USD)',  [1,2, 1.5, 2, 1.5,1,1,2, 1.5, 2,
                    1,2, 1.5, 2, 1.5,1]),
    ))
fav = DataFrame(table1)
fav

In [ ]:
fav.pivot_table(index='Person', columns='Item Purchased', values='Price Paid (USD)',aggfunc=np.sum)

Pivots are a specific form of stack/unstack (remember those?)

![pivot 1](http://nikgrozev.com/images/blog/Reshaping%20in%20Pandas%20-%20Pivot%20Pivot-Table%20Stack%20and%20Unstack%20explained%20with%20Pictures/stack-unstack1.png "pivots")

## A worked example

In [ ]:
# to start let's make a fake dataset: sales of fruit across US states.
# Don't worry about the details here, but basically we'll pretend
# this string is a CSV file and use the standard loading ops
from io import StringIO

TESTDATA=StringIO("""State,Retailer,Fruit,Sales
MI,Walmart,Apple,100
MI,Wholefoods,Apple,150
MI,Kroger,Orange,180
CA,Walmart,Apple,220
CA,Wholefoods,Apple,180
CA,Safeway,Apple,220
CA,Safeway,Orange,110
NY,Walmart,Apple,90
NY,Walmart,Orange,80
NY,Wholefoods,Orange,120
""")

fruit = pd.read_csv(TESTDATA, index_col=None)
fruit

## (a) What is the total sales for each state?
This requires us to group by state, and aggregate sales by taking the sum.

The easiest way of doing this if to use `groupby`

If you execute groupby on the dataframe what you'll get back is an object called DataFrameGroupBy

In [ ]:
fruit.groupby('State')

On its own it's a bit useless... it just keeps track of which rows should go into each "pile" (where pile here means a unique group for each state)

If we ask this object to describe itself, you can see what is inside notice that it threw away all the other columns because they were not numerical.  Only "Sales" which is a number, was kept

In [ ]:
fruit.groupby('State').describe()

Now, if we had another numerical column, let's call it "Sales2," that column would also be kept.  Let's make a fruit2 DataFrame so you can see that:

In [ ]:
from io import StringIO

TESTDATA=StringIO("""State,Retailer,Fruit,Sales,Sales2
MI,Walmart,Apple,100,10
MI,Wholefoods,Apple,150,20
MI,Kroger,Orange,180,30
CA,Walmart,Apple,220,20
CA,Wholefoods,Apple,180,40
CA,Safeway,Apple,220,30
CA,Safeway,Orange,110,20
NY,Walmart,Apple,90,40
NY,Walmart,Orange,80,20
NY,Wholefoods,Orange,120,60
""")

fruit2 = pd.read_csv(TESTDATA, index_col=None)
fruit2

In [ ]:
fruit2.groupby("State").describe()

To actually make use of the groupby, we need to tell pandas what to use to measure what's in each group. In other words, I've created a pile for California, a pile for Michigan, and a pile for New York.  I want a number to what's *inside* each pile.  I could ask for the "size" (so how many rows are in each pile), or I could calculate some mathematical function.  For example, if I wanted to know the total sales, I would call "sum."  What happens is pandas goes through every pile, looks at every "row" inside that pile and, for all numerical properties, calculated something.  In this case it's sum... it adds up everything.  So in our original table we had three items for Michigan (Walmart, Kroger, Wholefoods).  This is our Michigan pile.  We then look at numerical properties for Walmart, Kroger, and Wholefoods. In this case Sales.  Because we are using sum() that means add the sales of each.

In [ ]:
# What are the total sales for each state?
fruit.groupby('State').sum()  # instead of size()

What just happend? A couple of things:
- `groupby()` got first executed on `df`, returning an `DataFrameGroupBy` object. This object itself is useless unless coupled with an aggregation function, such as `sum()`, `mean()`, `max()`, `apply()`. 
- Then, `sum()` got executed on the `DataFrameGroupBy` object, generating the `DataFrame` object you see above. Notice how the table looks different than the original DataFrame `df`? Here are the differences:
  - The `State` column now becomes the index of the DataFrame. The string "State" is the name of the index. Notice how the index name is displayed on a lower level than column names.
  - Since we performed a `groupby` operation by `State`, so only the unique values of `State` are kept as index.
  - Among the other columns, Retailer, Fruit, and Sales, only Sales is kept in the result table. This is because the aggregation function `sum()` only knows how to aggregate numerical values. And only Sales is a numerical column. The other columns are hence dropped.

## <font color="magenta">Q11: (2 points) Using groupby, which Retailer had the highest total Sales and how much was that:

In [ ]:
# Insert code here

Insert written Answer here

## (b) What is the total sales for each state for each fruit?
This requires us to perform `groupby` on two columns. So, we provide a list of column names to the `groupby` function.

Don't forget that an aggregation function needs to follow the `groupby` function in order to generate results.

In [ ]:
# What is the total sales for each state for each fruit?
fruit.groupby(['State','Fruit']).sum()

How is this DataFrame different from the previous one?

The biggest different is that this DataFrame has what is called a `MultiIndex` (or hierarchical index), as opposed to a simple index. In this table, the left two "columns" are not columns but actually part of the `MultiIndex`, and the `Sales` is the single real "column" in the DataFrame. (Running out of terminologies here...)

The hierarchical index can be organized in an alternative way if we swapped the order of State and Fruit.

In [ ]:
fruit.groupby(['Fruit','State']).sum()

## (c) Which state has the maximum total sales?
This question is not asking about the maximum value, but rather which state holds that maximum. There are multiple ways to do it. A principled way is to use `idxmax`.

In [ ]:
# Which state has the maximum total sales?
fruitSalesByState = fruit.groupby('State').sum()
print(fruitSalesByState)
max_state = fruitSalesByState['Sales'].idxmax()
print("The state with the maximum sales is: ",max_state)



What if I want to display the maximum value alongside the state? Well, we can use that returned label to _select_ the corresponding row from the original DataFrame.

In [ ]:
# the deets
fruitSalesByState.loc['CA']

A less efficient but more intuitive way of doing the same thing:

In [ ]:
fruitSalesByState.sort_values('Sales',ascending=False).iloc[0]

### Which state has the maximum total sales for apples?</font>

In [ ]:
# Which state has the maximum total sales for apples?
# give me apple sellers
apples = fruit[fruit.Fruit == 'Apple']
apples

In [ ]:
# aggr. by state
applesByState = apples.groupby('State').sum()
applesByState

In [ ]:
applesByState.Sales

In [ ]:
applesByState.Sales.idxmax()

In [ ]:
applesByState.loc[applesByState.Sales.idxmax()]

In the above command, `.loc[]` looks up the index label and returns that row.



## Applying what we learned to the LOTR data

What are the average values for magic, aggression, and stealth for each race?

In [ ]:
from io import StringIO

LOTRDATA=StringIO("""name,race,gender,magic,aggression,stealth
Gandalf,Maia,Male,10.0,7.0,8.0
Gimli,Dwarf,None,1.0,10.0,2.0
Frodo,Hobbit,Male,4.0,2.0,5.0
Legolas,Elf,Male,6.0,5.0,10.0
Bilbo,Hobbit,Male,4.0,1.0,5.0
Sam,Hobbit,Male,2.0,6.0,4.0
Pippin,Hobbit,Male,0.0,3.0,5.0
Boromir,Human,Male,0.0,8.0,3.0
Aragorn,Human,Male,2.0,7.0,9.0
Galadriel,Elf,Female,9.0,2.0,10.0
Lily,Hobbit,Female,,,
Meriadoc,Hobbit,Male,,4.0,6.0
Melian,Maia,Female,10.0,5.0,9.0
Idril,Elf,Female,8.0,,8.0
""")

lotr = pd.read_csv(LOTRDATA, index_col=None)

In [ ]:
lotr.head()

## <font color="magenta">Q12: (2 points) Create a pivot table showing the maximum values of magic using gender for columns and race for rows.  

In [ ]:
#Insert Code Here

# <font color="magenta">END OF NOTEBOOK</font>
## Remember to submit this file in HTML and IPYNB formats via Canvas.